In [15]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['font.size'] = 10
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

선형 회귀 분석은 가장 기본적인 머신러닝 방법이며, 여러 분야에서 사용된다.

선형 회귀 분석은 피쳐 데이터와 레이블 간의 선형 관계를 파악하는 알고리즘으로 피쳐 데이터 x와 레이블 y 사이에 선형 관계가 존재할 때 이를 수식화 하면 다음과 같다.

$$y = wx + b$$

데이터 x가 주어질 때, 데이터 x에 가중치(Weight) w를 곱하고 바이어스 b를 더하면 레이블을 얻을 수 있다. 선형 회귀에서 해야할 일은 피쳐 데이터 x와 레이블 y를 이용해서 가중치 w와 바이어스 b를 구하는 것이다.

위의 수식은 피쳐 데이터의 개수가 1개일 경우이고, 이를 일반화시켜 p개의 피쳐 데이터를 가지는 데이터라고 가정하면 데이터 셋을 구성하는 각 데이터 포인트는 p개의 학습 데이터로 구성되므로 $x = (x_1, x_2, x_3, ..., x_p)$라고 표현할 수 있다. x<sub>i</sub>는 i번째 데이터 행을 열벡터로 표현한 것이다. 그리고 선형 회귀 모델은 아래와 같다.

$$y = \hat y = w_1x_1 + w_2x_2 + w_3x_3 + ... + w_px_p + b$$

이때, $w = (w_1, w_2, w_3, ..., w_p)^T$를 가중치라고 부른다. 각 가중치 요소 하나하나가 우리가 구하려는 파라미터이며, 파라미터 값은 예측에 영향을 미친다. 즉, 파라미터 값에 따라서 예측이 달라진다.

가중치는 학습 데이터로 부터 최소 제곱법(Least Suqares Etimator)을 사용해 구할 수 있다. 최소 제곱법은 오차의 제곱합이 최소가 되는 추정량을 구하는 방법이다.

보스턴 집값 데이터를 사용해 보스턴 집값을 예측하는 모델을 생성하고 학습시킨다.  
일반 선형 회귀 뿐만 아니라 라쏘(L1 제약식 사용), 릿지(L2 제약식 사용), 엘라스틱 넷을 사용한 결과도 비교해 본다.

In [2]:
# 데이터 불러오기
raw_data = datasets.load_boston() # 사이킷런이 제공하는 보스턴 집값 데이터를 불러온다.

# 피쳐, 레이블 데이터 저장
xData = raw_data.data # 피쳐 데이터를 저장한다.
yData = raw_data.target # 피쳐 데이터에 따른 레이블을 저장한다.
print(xData.shape, yData.shape)

# 학습 데이터와 테스트 데이터로 분할
x_train, x_test, y_train, y_test = train_test_split(xData, yData, random_state=1)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

# 데이터 표준화
std_scale = StandardScaler() # 표준화 스케일러 객체를 만든다.
x_train = std_scale.fit_transform(x_train) # 학습 데이터를 스케일러로 표준화 하고 적용한다.
x_test = std_scale.transform(x_test) # 테스트 데이터를 학습 데이터로 표준화 스케일러에 적용한다.

(506, 13) (506,)
(379, 13) (127, 13) (379,) (127,)


일반 선형 회귀 분석

In [3]:
# 모델 생성 후 데이터 학습
from sklearn.linear_model import LinearRegression # 선형 회귀 알고리즘을 사용하기 위해 import 한다.
# 선형 회귀 모델을 만든다.
clf_lr = LinearRegression()
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 일반 선형 회귀 모델을 학습시킨다.
clf_lr.fit(x_train, y_train)

LinearRegression()

In [4]:
# 일반 선형 회귀 분석 계수(가중치)와 상수항(바이어스) 확인
print(clf_lr.coef_) # 가중치
print(clf_lr.intercept_) # 바이어스

[-1.07145146  1.34036243  0.26298069  0.66554537 -2.49842551  1.97524314
  0.19516605 -3.14274974  2.66736136 -1.80685572 -2.13034748  0.56172933
 -4.03223518]
22.344591029023768


라쏘(Lasso) 선형 회귀 분석(L1 제약식 사용)

In [5]:
# 모델 생성 후 데이터 학습
from sklearn.linear_model import Lasso # 라쏘 선형 회귀 알고리즘을 사용하기 위해 import 한다.
# 라쏘 선형 회귀 모델의 alpha 속성으로 제약 정도를 지정해서 모델을 만든다.
# alpha 속성의 기본값은 1이고 반드시 양수로 지정해야 하며 값이 클 수록 강한 제약식을 의미한다.
clf_lasso = Lasso(alpha=0.01)
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 라쏘 선형 회귀 모델을 학습시킨다.
clf_lasso.fit(x_train, y_train)

Lasso(alpha=0.01)

In [6]:
# 라쏘 선형 회귀 분석 계수(가중치)와 상수항(바이어스) 확인
print(clf_lasso.coef_) # 가중치
print(clf_lasso.intercept_) # 바이어스

[-1.04326518  1.27752711  0.1674367   0.66758228 -2.41559964  1.99244179
  0.14733958 -3.09473711  2.46431135 -1.60552274 -2.11046422  0.55200229
 -4.00809905]
22.344591029023768


릿지(Ridge) 선형 회귀 분석(L2 제약식 사용)

In [7]:
# 모델 생성 후 데이터 학습
from sklearn.linear_model import Ridge # 릿지 선형 회귀 알고리즘을 사용하기 위해 import 한다.
# 릿지 선형 회귀 모델의 alpha 속성으로 제약 정도를 지정해서 모델을 만든다.
# alpha 속성의 기본값은 1이고 반드시 양수로 지정해야 하며 값이 클 수록 강한 제약식을 의미한다.
clf_ridge = Ridge(alpha=1)
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 릿지 선형 회귀 모델을 학습시킨다.
clf_ridge.fit(x_train, y_train)

Ridge(alpha=1)

In [8]:
# 릿지 선형 회귀 분석 계수(가중치)와 상수항(바이어스) 확인
print(clf_ridge.coef_) # 가중치
print(clf_ridge.intercept_) # 바이어스

[-1.05933451  1.31050717  0.23022789  0.66955241 -2.45607567  1.99086611
  0.18119169 -3.09919804  2.56480813 -1.71116799 -2.12002592  0.56264409
 -4.00942448]
22.344591029023768


엘라스틱 넷(Elastic Net) 선형 회귀 분석(L1, L2 제약식 모두 사용)

In [9]:
# 모델 생성 후 데이터 학습
from sklearn.linear_model import ElasticNet # 엘라스틱 넷 선형 회귀 알고리즘을 사용하기 위해 import 한다.
# 엘라스틱 넷 선형 회귀 모델의 alpha 속성으로 제약 정도를 지정해서 모델을 만든다.
# alpha 속성에는 L1 제약식의 크기와 L2 제약식의 크기의 합을 지정한다.
# l1_ratio 속성으로 alpha에 지정한 값에서 L1 제약이 차지하는 비율을 지정한다.
# l1_ratio 속성값은 비율이므로 0부터 1 사이의 값을 지정해야 하며, 0을 지정하면 L1 제약이 사용되지 않고
# L2 제약만 사용하는 릿지 선형 회귀 분석을 의미하고, 1을 지정하면 L2 제약이 사용되지 않고 L1 제약만
# 사용하는 라쏘 선형 회귀 분석이 됨을 의미한다.
clf_elastic = ElasticNet(alpha=0.01, l1_ratio=0.01)
# 표준화된 학습 데이터와 학습 데이터에 따른 레이블 데이터로 엘라스틱 넷 선형 회귀 모델을 학습시킨다.
clf_elastic.fit(x_train, y_train)

ElasticNet(alpha=0.01, l1_ratio=0.01)

In [10]:
# 엘라스틱 넷 선형 회귀 분석 계수(가중치)와 상수항(바이어스) 확인
print(clf_elastic.coef_) # 가중치
print(clf_elastic.intercept_) # 바이어스

[-1.02916603  1.23681955  0.15236504  0.67859622 -2.34646781  2.02965524
  0.14575132 -2.98592423  2.32013379 -1.48829485 -2.09271972  0.56506801
 -3.9495281 ]
22.344591029023768


학습된 모델로 테스트 데이터를 예측한다.

In [11]:
# predict() 메소드의 인수로 표준화된 테스트 데이터를 넘겨서 예측한다.
predict_lr = clf_lr.predict(x_test)
predict_lasso = clf_lasso.predict(x_test)
predict_ridge = clf_ridge.predict(x_test)
predict_elastic = clf_elastic.predict(x_test)
for i in range(len(predict_lr))[:5]:
    print('일반: {:8.5f}, 라쏘: {:8.5f}, 릿지: {:8.5f}, 엘라: {:8.5f}'.format(
        predict_lr[i], predict_lasso[i], predict_ridge[i], predict_elastic[i]
    ))

일반: 32.37817, 라쏘: 32.19249, 릿지: 32.26711, 엘라: 31.99335
일반: 27.95684, 라쏘: 27.98937, 릿지: 27.97205, 엘라: 28.00404
일반: 18.07214, 라쏘: 18.04237, 릿지: 18.08339, 엘라: 18.11898
일반: 21.63167, 라쏘: 21.75823, 릿지: 21.68419, 엘라: 21.81160
일반: 18.93030, 라쏘: 18.88655, 릿지: 18.89663, 엘라: 18.81388


학습된 모델을 평가한다.

In [12]:
# 평균 절대값 오차(MAE)
# mean_absolute_error() 메소드의 인수로 테스트 데이터의 레이블과 예측값을 넘겨 평균 절대값 오차를 계산한다.
print('일반:', mean_absolute_error(y_test, predict_lr))
print('라쏘:', mean_absolute_error(y_test, predict_lasso))
print('릿지:', mean_absolute_error(y_test, predict_ridge))
print('엘라:', mean_absolute_error(y_test, predict_elastic))

일반: 3.574868126127544
라쏘: 3.571868321373356
릿지: 3.5725770166994746
엘라: 3.5698270945059454


In [13]:
# 평균 제곱 오차(MAE)
# mean_squared_error() 메소드의 인수로 테스트 데이터의 레이블과 예측값을 넘겨 평균 제곱 오차를 계산한다.
print('일반:', mean_squared_error(y_test, predict_lr))
print('라쏘:', mean_squared_error(y_test, predict_lasso))
print('릿지:', mean_squared_error(y_test, predict_ridge))
print('엘라:', mean_squared_error(y_test, predict_elastic))

일반: 21.897765396049525
라쏘: 21.915483810504835
릿지: 21.894849212618745
엘라: 21.912961890936874


In [14]:
# R2 score
# r2_score() 메소드의 인수로 테스트 데이터의 레이블과 예측값을 넘겨 R2 score를 계산한다.
print('일반:', r2_score(y_test, predict_lr))
print('라쏘:', r2_score(y_test, predict_lasso))
print('릿지:', r2_score(y_test, predict_ridge))
print('엘라:', r2_score(y_test, predict_elastic))

일반: 0.7789410172622855
라쏘: 0.7787621490259894
릿지: 0.7789704562726605
엘라: 0.7787876079239252
